<a href="https://colab.research.google.com/github/kakiang/feed-scraper-cloud-functions/blob/master/Bible_in_Mundang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Bibel

In [618]:
%%writefile scrape_bible_dot_com.py
"""A program that scrapes 'https://www.bible.com/fr/bible/'."""

import csv
import pandas as pd
import logging
import os
import re
import uuid
from datetime import datetime
from time import mktime

import requests
from bs4 import BeautifulSoup

logging.basicConfig(format="%(asctime)s - %(message)s", datefmt="%d-%b-%y %H:%M:%S")
logger = logging.getLogger(__name__)

books = {
    "GEN": 50,
    "EXO": 40,
    "LEV": 27,
    "NUM": 36,
    "DEU": 34,
    "JOS": 24,
    "JUD": 1,
    "RUT": 4,
    "1SA": 31,
    "2SA": 24,
    "1KI": 22,
    "2KI": 25,
    "1CH": 29,
    "2CH": 36,
    "EZR": 10,
    "NEH": 13,
    "EST": 10,
    "JOB": 42,
    "PSA": 150,
    "PRO": 31,
    "ECC": 12,
    "SNG": 8,
    "ISA": 66,
    "JER": 52,
    "LAM": 5,
    "EZE": 48,
    "DAN": 12,
    "HOS": 14,
    "JOE": 3,
    "AMO": 9,
    "OBA": 1,
    "JON": 4,
    "MIC": 7,
    "NAH": 3,
    "HAB": 3,
    "ZEP": 3,
    "HAG": 2,
    "ZEC": 14,
    "MAL": 4,
    "MAT": 28,
    "MAR": 16,
    "LUK": 24,
    "JOH": 21,
    "ACT": 28,
    "JAM": 5,
    "1PE": 5,
    "2PE": 3,
    "1JO": 5,
    "2JO": 1,
    "3JO": 1,
    "ROM": 16,
    "1CO": 16,
    "2CO": 13,
    "GAL": 6,
    "EPH": 6,
    "PHI": 1,
    "COL": 4,
    "1TH": 5,
    "2TH": 3,
    "1TI": 6,
    "2TI": 4,
    "TIT": 3,
    "HEB": 13,
    "REV": 22,
}


def scrape_web_page(url):
    """Scrape a webpage and returns the soup."""
    page_response = requests.get(url, timeout=5)
    if page_response.status_code != 200:
        raise ValueError("page_response.status!=200")
    soup = BeautifulSoup(page_response.content, "html.parser")
    return soup


def get_verses(soup):
    """Get verses tags from soup."""
    if soup is None:
        raise ValueError("soup is None")
    list_verses = soup.find_all("span", class_=re.compile("^verse"))
    return list_verses


def parse_verses(result_set):
    """Parses verses tags and returns python list of verses."""
    list_verses_raw = list(result_set)
    list_books = list(books)
    list_size = len(list_verses_raw)
    verses = []
    i = 0
    while i < list_size:
        reference = list_verses_raw[i]["data-usfm"]
        ref = reference.split(".")
        book_index = list_books.index(ref[0]) + 1
        verse = {}
        text = ''
        while i < list_size and list_verses_raw[i]["data-usfm"] == reference:
            all_contents = list_verses_raw[i].find_all(class_='content')
            for content in all_contents:
                if content.string.strip():
                    text = f"{text} {content.string}".strip()
            i += 1
        if text:
            verse["id"] = f"{str(book_index).zfill(2)}{ref[1].zfill(3)}{ref[2].zfill(3)}"
            verse["reference"] = reference
            verse["book"] = book_index
            verse["chapter"] = ref[1]
            verse["verse"] = ref[2]
            verse["text"] = text
            verses.append(verse)
    return verses


def save_to_file(data, header, file_name="bible_output.csv"):
    """Save the scraped data to a file."""
    if data is None:
        raise ValueError("data is None")
    with open(file_name, "wt") as out_file:
        writer = csv.DictWriter(out_file, fieldnames=header)
        writer.writeheader()
        writer.writerows(data)
    return file_name


def main():
    """Defines program entry point."""
    data = []
    for key, value in books.items():
        print('\n', key, end=':')
        for i in range(1, value + 1):
            print(i, end=',')
            # url = f"https://www.bible.com/fr/bible/863/{key}.{i}.MUAR"
            url = f"https://www.bible.com/fr/bible/93/{key}.{i}.LSG"
            soup = scrape_web_page(url)
            result_set = get_verses(soup)
            parsed_list_verses = parse_verses(result_set)
            if len(parsed_list_verses) > 0:
                data.extend(parsed_list_verses)
    # header = ["id", "reference", "book", "chapter", "verse", "text"]
    # file_name = save_to_file(data, header, file_name="bible_french.csv")
    scraped_df = pd.DataFrame(data)
    scraped_df.to_csv('bible_french_lsg.csv', index=False)
    print("\nData successfully saved")


# if __name__ == "__main__":
#     main()

Writing scrape_bible_dot_com.py


In [ ]:
# main()
# url = f"https://www.bible.com/fr/bible/93/GEN.1.LSG"
# soup = scrape_web_page(url)
# result_set = get_verses(soup)
# result_set

In [1]:
# from google.colab import files
# uploaded = files.upload()
# ! mv bible_mundang\ \(3\).csv ./bible_mundang.csv

In [97]:
import re
from itertools import chain
import pandas as pd
import numpy as np
pd.set_option('display.width', 100)
pd.set_option('max_rows', 100)
pd.set_option('max_colwidth', 500)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [603]:
df_f.to_csv('bible_french_lsg.csv', index=False)
df_f = pd.read_csv('bible_french_lsg.csv')
print(df_f.shape)
df_f.head()

,id,reference,book,chapter,verse,text
0,01001001,GEN.1.1,1,1,1,"Au commencement, Dieu créa les cieux et la terre."
1,01001002,GEN.1.2,1,1,2,"La terre était informe et vide: il y avait des ténèbres à la surface de l’abîme, et l’esprit de Dieu se mouvait au-dessus des eaux."
2,01001003,GEN.1.3,1,1,3,Dieu dit: Que la lumière soit! Et la lumière fut.
3,01001004,GEN.1.4,1,1,4,Dieu vit que la lumière était bonne; et Dieu sépara la lumière d’avec les ténèbres.
4,01001005,GEN.1.5,1,1,5,"Dieu appela la lumière jour, et il appela les ténèbres nuit. Ainsi, il y eut un soir, et il y eut un matin: ce fut le premier jour."


In [549]:
# df_m.to_csv('bible_mundang.csv', index=False)
df_m = pd.read_csv('bible_mundang.csv')
print(df_m.shape)
df_m.head()

(26401, 6)


,id,reference,book,chapter,verse,text
0,1001001,GEN.1.1,1,1,1,"Ne cok Masǝŋ mo tǝŋ joŋ sǝŋ ne sǝrri ,"
1,1001002,GEN.1.2,1,1,2,"ka sǝr ah zyeɓ vǝr a ba, a tǝkolle. Bii no cok daŋ, so cokfuu no tǝtǝl ahe, Tǝ'yak Masǝŋ a kiŋ tǝ bii ahe."
2,1001003,GEN.1.3,1,1,3,"Masǝŋ faa ka cokfãi mo joŋ, so cokfãi joŋni."
3,1001004,GEN.1.4,1,1,4,"Masǝŋ kwo cokfãi ah pǝsãhe, so woŋ kǝsyil cokfãi ne cokfuu."
4,1001005,GEN.1.5,1,1,5,"Masǝŋ so ɗii cokfãi ah ne comme, ɗii cokfuu ah ne suŋni. So lil joŋni, zah'nan so joŋni, zah'nan vaŋno ah yo."


In [231]:
# deletes = df_m[df_m.id.astype(np.str).str.contains(re.compile('[A-Za-z]'))].index
# deletes = df_m[df_m.verse.str.contains(re.compile('[A-Za-z]'))].index
# list(deletes)
# df_m.drop(deletes, inplace=True)
# df_m[df_m.verse.str.contains(re.compile('[A-Za-z]'))]
# dupl = df_m[df_m.verse.str.contains(re.compile('[A-Za-z]'))]
# dupl.reset_index(inplace=True, drop=True)

In [232]:
# ref_list = list(chain.from_iterable([ref.split('+') for ref in dupl.reference.values]))

In [233]:
# df_f_del = df_f[df_f.reference.isin(ref_list)].index
# df_f_del

In [234]:
# df_f.drop(df_f_del, inplace=True)

In [235]:
# df_f[df_f.reference.isin(ref_list)]

In [236]:
# other = []

In [ ]:
# lev64 = {"id":"03006004", "reference":"LEV.6.4", "book":3, "chapter":6, "verse":4, "text": "Madǝwin ye ɓe, lak ah solai vãm jemma sai."}
# lev64 = {"id":"03006005", "reference":"LEV.6.5", "book":3, "chapter":6, "verse":5, "text": "Madǝwin ye ɓe, lak ah solai vãm jemma sai."}
# lev820 = {"id":"03008020", "reference":"LEV.8.20", "book":3, "chapter":8, "verse":20, "text": "Madǝwin ye ɓe, lak ah solai vãm jemma sai."}
# lev821 = {"id":"03008021", "reference":"LEV.8.21", "book":3, "chapter":8, "verse":21, "text": "Madǝwin ye ɓe, lak ah solai vãm jemma sai."}


# lev272 = {"id":"03027002", "reference":"LEV.27.2", "book":3, "chapter":27, "verse":2, "text": "Mo faa nyi za Israel sye: Dǝɓ mo gbǝ zah pel Dǝɓlii ka nyi dǝfuu nyi ko, dǝɓ ah gak wǝǝ tǝtǝl dǝɓ ah ne lak tǝgbana ɓǝ lai ah mo cuu ɓo, tǝgbana 'man lak ah mo kanra ɓo:"}
# lev273 = {"id":"03027003", "reference":"LEV.27.3", "book":3, "chapter":27, "verse":3, "text": "Dǝwor mai mo joŋ syii ɓo jemma gwa ŋhaa dai syii jemma yea, lak ah solai vãm jemma dappe."}
# lev274 = {"id":"03027004", "reference":"LEV.27.4", "book":3, "chapter":27, "verse":4, "text": "Madǝwin ye ɓe, lak ah solai vãm jemma sai."}
# lev275 = {"id":"03027005", "reference":"LEV.27.5", "book":3, "chapter":27, "verse":5, "text": "We wor ma joŋ syii dappe ŋhaa dai syii jemma gwa ye ɓe, lak ah solai vãm jemma gwa, Mǝlaŋ win ye ɓe, lak ah solai vãm jemma vaŋno."}
# lev276 = {"id":"03027006", "reference":"LEV.27.6", "book":3, "chapter":27, "verse":6, "text": "Mo we wor mai mo joŋ fĩi ɓo vaŋno ŋhaa dai syii dappe ye ɓe, lak ah solai vãm dappe, Mo mǝlaŋ win o, lak ah solai vãm sai."}
# lev277 = {"id":"03027007", "reference":"LEV.27.7", "book":3, "chapter":27, "verse":7, "text": "Mo dǝwor ma joŋ syii tǝgbana jemma yea wala ma kal ah ye ɓe, lak ah solai vãm jemma tǝtǝl dappe, Mo madǝwin ye mo joŋ syii ɓo tǝgbana jemma yea wala ma kal ah ye ɓe, lak ah solai vãm jemma vaŋno."}

In [ ]:
# num10 = {"id":"04036010", "reference":"NUM.36.10", "book":4, "chapter":36, "verse":10, "text": "Wee Selofad maŋwǝǝ joŋra tǝgbana mai Dǝɓlii mo faa nyi Mosus."}
# num11 = {"id":"04036011", "reference":"NUM.36.11", "book":4, "chapter":36, "verse":11, "text": "So Maka, Tersa, Milka, ne Noa kanra wee naa pah pa ɓǝǝra."}

# king1 = {"id":"11022036", "reference":"1KI.22.36", "book":11, "chapter":22, "verse":36, "text": "Ne cok com mo dan o, dahra gor zahjul sal faa: Zune daŋ mo pii soo ge yaŋ ahe, zune daŋ mo pii soo ge sǝr ahe, "}
# king2 = {"id":"11022037", "reference":"1KI.22.37", "book":11, "chapter":22, "verse":37, "text": "Mor goŋ wǝ ɓe. So ɓaŋra ko ge yaŋ Samaria ne ko, ge ciira ko gŋ."}

# other.extend([lev272,lev273,lev274,lev275,lev276,lev277,num10,num11,king1,king2])

In [ ]:
# EXO1633 = {"id":"01016033", "reference":"EXO.16.33", "book":2, "chapter":16, "verse":33, "text": "Mosus faa nyi Aron: Mo ɓaŋ ciiri, ka mo rǝk manna tahsah tǝ vaŋno ge gŋ, mo kan pel Dǝɓlii ka byak gŋ mor wee ɓii ma fahfalle."}
# EXO1634 = {"id":"01016034", "reference":"EXO.16.34", "book":2, "chapter":16, "verse":34, "text": "Aron ge kan fan ah pel sunduku gbanzahe, tǝgbana Dǝɓlii mo faa."}
# EXO1635 = {"id":"01016035", "reference":"EXO.16.35", "book":2, "chapter":16, "verse":35, "text": "Za Israel rera manna tǝgǝǝ syii jemma nai ŋhaa ge daira tǝsyee sǝr Kanaan."}
# EXO1636 = {"id":"01016036", "reference":"EXO.16.36", "book":2, "chapter":16, "verse":36, "text": "Tahsah ah vaŋno a tǝgbana tahsah jemma gwa ma tǝ'nahko."}
# EXO191 = {"id":"01019001", "reference":"EXO.16.36", "book":2, "chapter":19, "verse":1, "text": "Ne patǝ sai fĩi fahfal za Israel mo pǝ̃ǝra gin sǝr Egib, ge daira kǝsyicok Sinai."}
# EXO192 = {"id":"01019002", "reference":"EXO.16.36", "book":2, "chapter":19, "verse":2, "text": "Urra gin Refidim. Ɗǝrra gŋ kah waare."}

# EXO2714 = {"id":"01027014", "reference":"EXO.27.15", "book":2, "chapter":27, "verse":14, "text": "Zyim rii cok ah ma fah kǝ zahfah kǝki ah mo i jol jemma tǝ dappe, wǝǝdǝǝ ah ra mo yea sai, ɓal ah ra mo yea sai ta."}
# EXO2715 = {"id":"01027015", "reference":"EXO.27.15", "book":2, "chapter":27, "verse":15, "text": "Kǝki ah a nai ta."}

# other.extend([EXO1633,EXO1634,EXO1635,EXO1636,EXO191,EXO192,EXO2714,EXO2715])

In [600]:
# df_f.id = df_f.id.astype(np.str).apply(lambda el: el.zfill(8))

In [296]:
# print(df_m.shape, df_f.shape)

(26448, 6) (25423, 6)


In [385]:
# r1 = 6953
# r2 = 6956 
# df_f[r1:r2]
# df_m[r1:r2]

,id,reference,book,chapter,verse,text
6953,9024020,1SA.24.20,9,24,20,"Si quelqu’un rencontre son ennemi, le laisse-t-il poursuivre tranquillement son chemin? Que l’Éternel te récompense pour ce que tu m’as fait en ce jour!"
6954,9024021,1SA.24.21,9,24,21,"Maintenant voici, je sais que tu régneras, et que la royauté d’Israël restera entre tes mains."
6955,9024022,1SA.24.22,9,24,22,"Jure-moi donc par l’Éternel que tu ne détruiras pas ma postérité après moi, et que tu ne retrancheras pas mon nom de la maison de mon père."


,id,reference,book,chapter,verse,text
6953,9024019,1SA.24.19,9,24,19,"Dǝɓ mo lwaa pa syiŋ ah ge pel ɓe, a soɓ ko kal jam ne? Dǝɓlii mo soo mo tǝ ɓǝ sãh mai mo joŋ ɓo wo ɓe tǝ'nahko."
6954,9024020,1SA.24.20,9,24,20,"Daga zǝzǝ̃ǝko me tǝ ɓe, mo ye ga yea goŋe. Goŋ Israel ga yea jol ɓo."
6955,9024021,1SA.24.21,9,24,21,"Mo haa zah nyi me ne tǝɗii Dǝɓlii, mo ka mgboo tǝ nah ɓe fahfal ɓe ya, so mo ka muŋ tǝɗii ɓe gin yaŋ pa ɓe ya."


In [585]:
# df_f.drop(list(range(10983,11078)), inplace=True)

In [69]:
# gen14 = {"id":"01009014", "reference":"GEN.9.14", "book":1, "chapter":9, "verse":14, "text": "Ne cok mee ga tai swãh sǝŋ tǝcee moo ga pǝ̃ǝ pǝzyil ah "}
# gen15 = {"id":"01009015", "reference":"GEN.9.15", "book":1, "chapter":9, "verse":15, "text": "me ga foo ɓǝ gbanzah ɓe kǝsyil ɓe ne we tǝkine fan ma ne cee daŋ. Bii ka ga ge tǝ sǝr ka vǝr fan ma ne cee faɗa yao."}

# gen29 = {"id":"01024029", "reference":"GEN.24.29", "book":1, "chapter":24, "verse":29, "text": "Rebeka no ne naa mam pǝworre, a ɗii ne Laban."}
# gen30 = {"id":"01024030", "reference":"GEN.24.30", "book":1, "chapter":24, "verse":30, "text": "Laa ɓǝ mai Rebeka mo kee nyi ra, tǝkine ɓǝ mai dǝɓ yeɓ Abraham mo faa nyi ko daŋ, so kwo tǝgaa ne fan zahkpem jol mo zahkpem jol naa mah ahe. So Laban pǝ̃ǝ ɗuu kal ge wo dǝɓ ah ka uu ɓo kah njoŋnjoŋ ah ra zah lak bii."}

# gen3610 = {"id":"01036010", "reference":"GEN.36.10", "book":1, "chapter":36, "verse":10, "text": "Esau byaŋ we vaŋno ne Ada, a ɗii ne Elifas. Esau so byaŋ we maki ah ne Basemat, a ɗii ne Ruel."}
# gen3611 = {"id":"01036011", "reference":"GEN.36.11", "book":1, "chapter":36, "verse":11, "text": "Elifas bem wee dappe, tǝɗii ɓǝǝ naiko: Teman ne Omar, Zefo, Gatam, tǝkine Kenas."}
# gen3612 = {"id":"01036012", "reference":"GEN.36.12", "book":1, "chapter":36, "verse":12, "text": "Esau byaŋ we maki ah ne Timna, a ɗii ne Amalek."}
# gen3613 = {"id":"01036013", "reference":"GEN.36.13", "book":1, "chapter":36, "verse":13, "text": "Ruel bem wee nai, tǝɗii ɓǝǝ naiko: Nahat, Zera, Syamma, tǝkine Mizza."}

# gen4321 = {"id":"01043021", "reference":"GEN.43.21", "book":1, "chapter":43, "verse":21, "text": "ne cok ru ge dai pǝ cok ɗǝrri, ru gbǝr zah dah ɓuuru, ru lwaa lak zune daŋ maa ɓo zah dah ahe, lak zune daŋ na kee ah mo kee ɓo. Ru so woo jin ge ɓo ne ko."}
# gen4322 = {"id":"01043022", "reference":"GEN.43.22", "book":1, "chapter":43, "verse":22, "text": "Ru so woo lak maki ah ge ɓo ne ka lee farel ne ta. Ru tǝ dǝɓ mai mo ge rǝk lak ɓuu zah dah ɓuur a."}

# # other.extend([gen14,gen15,gen29,gen30,gen3610,gen3611,gen3612,gen3613,gen4321,gen4322])

In [ ]:
# ezr31 = {"id":"15010031", "reference":"EZR.10.31", "book":15, "chapter":10, "verse":31, "text": "Ma morsǝ̃ǝ Harim ye Eliezar, Isija, Malkia, Semaja, Simon,"}
# ezr32 = {"id":"15010032", "reference":"EZR.10.32", "book":15, "chapter":10, "verse":32, "text": "Benyaamin, Maluk, ne Semaria."}
# ezr34 = {"id":"15010034", "reference":"EZR.10.34", "book":15, "chapter":10, "verse":34, "text": "Ma morsǝ̃ǝ Bani ye Maadai, Amram, Uel,"}
# ezr35 = {"id":"15010035", "reference":"EZR.10.35", "book":15, "chapter":10, "verse":35, "text": "Benaja, Bedia, Keluku,"}
# ezr36 = {"id":"15010036", "reference":"EZR.10.36", "book":15, "chapter":10, "verse":36, "text": "Vania, Merimot, Eliasiɓ,"}
# ezr37 = {"id":"15010037", "reference":"EZR.10.37", "book":15, "chapter":10, "verse":37, "text": "Mattania, Matenai, ne Jaasai."}
# ezr38 = {"id":"15010038", "reference":"EZR.10.38", "book":15, "chapter":10, "verse":38, "text": "Bani, Binwi, Syimai,"}
# ezr39 = {"id":"15010039", "reference":"EZR.10.39", "book":15, "chapter":10, "verse":39, "text": "Selemia, Natan, Adaija,"}
# ezr40 = {"id":"15010040", "reference":"EZR.10.40", "book":15, "chapter":10, "verse":40, "text": "Maknabai, Saskai, Syarai,"}
# ezr41 = {"id":"15010041", "reference":"EZR.10.41", "book":15, "chapter":10, "verse":41, "text": "Azarel, Selemia, Semaria,"}
# ezr42 = {"id":"15010042", "reference":"EZR.10.42", "book":15, "chapter":10, "verse":42, "text": "Sallum, Amaria, ne Yuseɓ."}

# other.extend([ezr31,ezr32,ezr34,ezr35,ezr36,ezr37,ezr38,ezr39,ezr40,ezr41,ezr42])

In [ ]:
# jer21 = {"id":"24052021", "reference":"JER.52.21", "book":24, "chapter":52, "verse":21, "text": "Wǝǝdǝǝ matǝ gwa, pǝzyil ah gǝǝgǝǝ vaŋno ah i jol ɓo jemma tǝtǝl nama, suu waddǝǝ malii kahlaa ah i jol jemma tǝtǝl gwa, yǝk vãm ah 'ah ah nǝn tǝwee jol nai."}
# jer22 = {"id":"24052022", "reference":"JER.52.22", "book":24, "chapter":52, "verse":22, "text": "Tǝtǝl ma vãm syẽ ah mai mo tǝtǝl ahe, wah ah i jol dappe. Fan ma kǝkaŋ ah ne foto kpuu grenadin kiŋ tǝtǝl ah zyaŋ cyõ, fan ah ra daŋ a ne vãm syẽ."}

# other.extend([jer21,jer22])
# print(len(other))

In [70]:
# df_m = df_m.append(other, ignore_index=True)

In [602]:
# df_f = df_f.sort_values(by='id', ignore_index=True)
# sorted.reset_index(inplace=True, drop=True)
# df_f[2831:2840]

In [556]:
# df_f.drop([10973], inplace=True)
# df_m[df_m.duplicated(subset=['reference'], keep=False)]
# df_m[2852:2855]

In [607]:
# df_m.drop_duplicates(subset=['reference'], inplace=True, ignore_index=True)
df_f[df_f.reference == 'EZR.7.8+EZR.7.9']

,id,reference,book,chapter,verse,text
11031,150078+EZR,EZR.7.8+EZR.7.9,15,7,8+EZR,Urra gin sǝr Babilon ne fĩi patǝ vaŋno ah ne zah'nan vaŋno ta. Masǝŋ gbah jol ɓǝǝ ge daira yaŋ Jerusalem ne fĩi patǝ dappe ah ne zah'nan vaŋno ahe.


In [547]:
# !rm bible_french_lsg.csv bible_mundang.csv

In [610]:
df_ct = pd.concat([df_f[['text']][:11031], df_m[['text']][:11031]], axis=1, )
df_ct.head()

In [614]:
# df_ct.to_csv('fr_mu.tsv', sep='\t', header=None, index=False)

## Different rows

In [604]:
df_c = pd.concat([df_f[['reference']], df_m[['reference']]], axis=1, )

In [605]:
df_c.columns = ['fr', 'mo']

In [616]:
df_c[df_c.fr != df_c.mo][:10]

In [617]:
# f, l = 7917, 7935, 7950
f, l = 11030, 11032 # 11102
df_f[f:l]
df_m[f:l]

In [499]:
# for i in range(10973,10399):
#     df_m.loc[i,'id'] = df_f.loc[i,'id']
#     df_m.loc[i,'reference'] = df_f.loc[i,'reference']
#     df_m.loc[i,'book'] = df_f.loc[i,'chapter']
#     df_m.loc[i,'verse'] = df_f.loc[i,'verse']